<a href="https://colab.research.google.com/github/amitca71/colab/blob/main/pdfreader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf

In [ ]:
from pypdf import PdfReader
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
import os
import re
import pandas as pd
pdf_folder_path = f'{root_dir}/charles_data/'
os.listdir(pdf_folder_path)

['RFI 0800 - Louver Support Framing at Loading Dock Soffit (RESPONSE without attachment).pdf',
 'RFI 0500-MKA Response.pdf',
 'RFI 0502-Response.pdf']

In [ ]:
#rfi_date has two values for the sake of the example. actually, the 2nd one covers the 1st one and can be left alone
#initially, only the 1st one was there and rfi_date was not extracted from RFI 0800 - Louver Support Framing at Loading Dock Soffit (RESPONSE without attachment).pdf
#see how it was debugged below
regex_str_dct={
    "rfi_title": [r"(?P<rfi_title>(.|\n)*)REQUEST FOR INFORMATION" ],
    "rfi_number": [r"(.|\n)*REQUEST FOR INFORMATION (?P<rfi_number>\d+)"],
    "rfi_date": [r"(.|\n)*DATE : (?P<rfi_date>\d+/\d+/\d+)", r"(.|\n)*DATE\s*:\s*(?P<rfi_date>\d+/\d+/\d+)"],
    "project_name": [r"(.|\n)*PROJECT:\n(?P<project_name>.*)"],
    "due_date": [r"(.|\n)*Response Required By:  \n(?P<due_date>\d+/\d+/\d+)"]
}

# "rfi_date": [r"(.|\n)*DATE : (?P<rfi_date>\d+/\d+/\d+)", r"(.|\n)*DATE\s*:\s*(?P<rfi_date>\d+/\d+/\d+)"],

In [ ]:
def get_value_by_regex(page, regex_str):
  return (re.match(regex_str, page))

In [ ]:
rfis_meta_list=[]
for fl in os.listdir(pdf_folder_path):
  rfi_meta_dict={}
  pdf_file_path=f'{pdf_folder_path}/{fl}'
  pdf_reader=PdfReader(pdf_file_path)
  first_page_str=pdf_reader.pages[0].extract_text()
  for r in regex_str_dct.keys():
    for i in range (0, (len(regex_str_dct[r]))):
      matches=get_value_by_regex(first_page_str, regex_str_dct[r][i])
      if (matches):
#       print(matches.group(r))
        rfi_meta_dict[r]=matches.group(r)
        break;
#  print(rfi_meta_dict)
  if (len(rfi_meta_dict.keys())<len(regex_str_dct.keys())):
    print (f"file {pdf_file_path} does not contain all attributes")
    print("Missing values:", (set(regex_str_dct.keys()).difference(rfi_meta_dict.keys())))
  rfi_meta_pd=pd.DataFrame.from_dict(rfi_meta_dict,orient='index').T
  rfis_meta_list.append(rfi_meta_pd)

In [ ]:
full_pd=pd.concat(rfis_meta_list)
full_pd

,rfi_title,rfi_number,rfi_date,project_name,due_date
0,Louver Support Framing at Loading \nDock Soffit,0800,7/28/2016,11414The Collection,8/4/2016
0,EE - Step on Sheet S-112B.1,0500,12/31/2013,11209Ala Moana Center - Ewa Mall Expansion,1/7/2014
0,EE - Slab Depression on Detail A6/A-474,0502,1/2/2014,11209Ala Moana Center - Ewa Mall Expansion,1/9/2014


In [ ]:
full_pd.to_csv('/tmp/all_data.csv')

In [ ]:
!cat /tmp/all_data.csv

,rfi_title,rfi_number,rfi_date,project_name,due_date
0,"Louver Support Framing at Loading 
Dock Soffit",0800,7/28/2016,11414The Collection,8/4/2016
0,EE - Step on Sheet S-112B.1,0500,12/31/2013,11209Ala Moana Center - Ewa Mall Expansion,1/7/2014
0,EE - Slab Depression on Detail A6/A-474,0502,1/2/2014,11209Ala Moana Center - Ewa Mall Expansion,1/9/2014


In [ ]:
rfi_meta_dict={}
pdf_file_path="/content/gdrive/My Drive//charles_data//RFI 0800 - Louver Support Framing at Loading Dock Soffit (RESPONSE without attachment).pdf"
pdf_reader=PdfReader(pdf_file_path)
first_page_str=pdf_reader.pages[0].extract_text()
print(first_page_str)
for r in regex_str_dct.keys():
  for i in range (0, (len(regex_str_dct[r]))):
    matches=get_value_by_regex(first_page_str, regex_str_dct[r][i])
    if (matches):
      print(matches.group(r))
      rfi_meta_dict[r]=matches.group(r)
      break;
print(rfi_meta_dict)

Louver Support Framing at Loading 
Dock SoffitREQUEST FOR INFORMATION 0800
201 Merchant Street
Honolulu, Hawaii   96813
Tel: (808) 739-4391 Fax: (808) 532-3618
Pappageorge Haymes PartnersNUMBER:DATE :7/28/2016
RFI TO:DISCIPLINE: StructuralPROJECT:
11414The Collection
ATTACHMENTS:
Joel Pankros
640 North LaSalle Street
Chicago, Illinois   60654ORIGINATOR :To Architect: 2 Sheets
Response Required By:  
8/4/2016
QUESTION: Ray Milner Authored By:
Reference the attached sketches from shts. A404 and A 618.
A louver with steel support framing is shown at the loading dock soffit on sheets A 404 and A 618, see attached.  Please provide 
structural details for the steel support framing.
ANSWER:
[9/27/2016 9:36 AM Pappageorge Haymes Partners - Joel Pankros]
Refer to details (see attached) provided by Baldridge & Associates Structural Engineering, Inc (BASE) by the request of HDCC 
for support of louver L26, L27, LV28 & LV29By: Joel Pankros  - Pappageorge Haymes Partners 9/27/2016
DIRECTION TO SUB/